# Hands-on Beispiel LLM (2)

### 2. Fine-tuning - Anpassung an juristische Fachtexte
##### --- Juristische Fragen an ein fine-tuned Modell (Lokale LLM)

In diesem Abschnitt fine-tunen wir das Modell `dbmdz/german-gpt2` und stellen ihm die gleichen zwei juristischen Fragen zum AI Act wie im Baseline-Notebook.

Ziel ist es, dass das feingetunte Modell (llm-2) nun fundiertere und korrekte Antworten liefert.


In [1]:
# falls noch nicht installiert 

import sys
# !{sys.executable} -m pip install transformers datasets
# !{sys.executable} -m pip install 'accelerate>=0.26.0'


In [1]:
import torch
from transformers import AutoTokenizer, AutoModelForCausalLM, Trainer, TrainingArguments, DataCollatorForLanguageModeling
from datasets import Dataset

/Users/minye/Library/Python/3.9/lib/python/site-packages/urllib3/__init__.py:34: NotOpenSSLWarning: urllib3 v2.0 only supports OpenSSL 1.1.1+, currently the 'ssl' module is compiled with 'LibreSSL 2.8.3'. See: https://github.com/urllib3/urllib3/issues/3020
  warnings.warn(
/Users/minye/Library/Python/3.9/lib/python/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [13]:
# Lade Modell und Tokenizer (das Basis-Modell bleibt identisch)
model_name = "dbmdz/german-gpt2" 
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForCausalLM.from_pretrained(model_name)
model = model.bfloat16()

# Da GPT-2-Modelle oft keinen expliziten Padding-Token besitzen, setzen wir hier den EOS-Token als Padding-Token.
tokenizer.pad_token = tokenizer.eos_token

# Konfiguriere pad_token_id im Modell 
# (braucht man, wenn das Modell noch nicht standardmäßig für den Umgang mit dem Padding-Token eingestellt ist)
model.config.pad_token_id = tokenizer.eos_token_id
model.generation_config.pad_token_id = tokenizer.pad_token_id


## Domänenspezifischer Datensatz: Auszüge aus dem AI Act
Wir extrahieren zwei wichtige Absätze aus dem AI Act, die juristische Fachtermini und Anforderungen beinhalten. 

Hinweis: Die folgenden Textabschnitte sind exemplarisch und basieren auf öffentlich zugänglichen Informationen zum AI Act, z.B.: "https://eur-lex.europa.eu/legal-content/DE/TXT/?uri=CELEX:32024R1689"

In [3]:
# Ausgewählte Absätze aus dem AI Act (Beispiele)
ai_act_texts = [
    "Artikel 1 – Anwendungsbereich: 
    Diese Verordnung gilt für KI-Systeme, die in der Europäischen Union in Verkehr gebracht oder in Betrieb genommen werden, und legt die grundlegenden Anforderungen an Sicherheit, Transparenz und Verantwortlichkeit fest.",
    "Artikel 2 – Risikoklassifizierung: 
    KI-Systeme werden in Abhängigkeit von ihrem potenziellen Risiko in verschiedene Kategorien eingeteilt. Hochrisiko-KI-Systeme unterliegen strengen Anforderungen an ihre Konzeption, Entwicklung und den Betrieb, um die Sicherheit und den Schutz der Grundrechte zu gewährleisten."
]

In [14]:
# Erstelle ein Dataset aus den Auszügen
data_dict = {"text": ai_act_texts}
dataset = Dataset.from_dict(data_dict)
print("Domänenspezifischer Datensatz erstellt:")
print(dataset)

Domänenspezifischer Datensatz erstellt:
Dataset({
    features: ['text'],
    num_rows: 2
})


In [15]:
# %% [code]
# Tokenisiere den Datensatz
def tokenize_function(example):
    return tokenizer(example["text"], truncation=True, padding="max_length", max_length=256)

tokenized_dataset = dataset.map(tokenize_function, batched=True)
tokenized_dataset = tokenized_dataset.remove_columns(["text"])
print("Tokenisierter Datensatz:")
print(tokenized_dataset)

Map: 100%|██████████████████████████████████████████████████████████████████████████████████████████| 2/2 [00:00<00:00, 100.07 examples/s]

Tokenisierter Datensatz:
Dataset({
    features: ['input_ids', 'attention_mask'],
    num_rows: 2
})


In [6]:
# Erstelle einen DataCollator für das Language Modeling
data_collator = DataCollatorForLanguageModeling(tokenizer=tokenizer, mlm=False)

In [7]:
# Definiere Trainingsargumente – das Fine-Tuning erfolgt exemplarisch über wenige Epochen
training_args = TrainingArguments(
    output_dir="./llm_ai_act_finetuned",
    overwrite_output_dir=True,
    num_train_epochs=3,
    per_device_train_batch_size=1,
    save_steps=5,
    save_total_limit=2,
    logging_steps=1,
    learning_rate=5e-5,
    weight_decay=0.01,
)

In [8]:
# Initialisiere den Trainer für das Fine-Tuning
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_dataset,
    data_collator=data_collator,
)

In [21]:

# Starte das Fine-Tuning
print("Starte das Fine-Tuning mit AI Act-Daten...")
trainer.train()
print("Fine-Tuning abgeschlossen.")

# Modell auf die CPU schieben und und alle Eingaben auf der CPU verarbeiten 
model.to("cpu")


Starte das Fine-Tuning mit AI Act-Daten...


Step,Training Loss
1,2.160700
2,2.926000
3,2.088200
4,2.877900
5,2.033700
6,2.870800


Fine-Tuning abgeschlossen.


GPT2LMHeadModel(
  (transformer): GPT2Model(
    (wte): Embedding(50265, 768)
    (wpe): Embedding(1024, 768)
    (drop): Dropout(p=0.0, inplace=False)
    (h): ModuleList(
      (0-11): 12 x GPT2Block(
        (ln_1): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
        (attn): GPT2Attention(
          (c_attn): Conv1D(nf=2304, nx=768)
          (c_proj): Conv1D(nf=768, nx=768)
          (attn_dropout): Dropout(p=0.0, inplace=False)
          (resid_dropout): Dropout(p=0.0, inplace=False)
        )
        (ln_2): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
        (mlp): GPT2MLP(
          (c_fc): Conv1D(nf=3072, nx=768)
          (c_proj): Conv1D(nf=768, nx=3072)
          (act): NewGELUActivation()
          (dropout): Dropout(p=0.0, inplace=False)
        )
      )
    )
    (ln_f): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
  )
  (lm_head): Linear(in_features=768, out_features=50265, bias=False)
)

## Test: Juristische Fragen erneut stellen
Nun stellen wir wieder dieselben Fragen wie in llm-1, um zu prüfen, ob das feingetunte Modell (llm-2) bessere Antworten liefert.

In [22]:
def ask_question(prompt):
    input_ids = tokenizer.encode(prompt, return_tensors="pt")
    output = model.generate(input_ids, max_length=150, temperature=0.7, do_sample=True)
    generated_text = tokenizer.decode(output[0], skip_special_tokens=True)
    return generated_text

In [23]:
questions = [
    "Welche Anforderungen stellt der AI Act an Hochrisiko-KI-Systeme?",
    "Was versteht man unter Transparenz gemäß dem AI Act?"
]

In [25]:
print("\n=== llm-2: Feingetuntes Modell ===\n")
for q in questions:
    print("Frage:", q)
    print() 
    answer = ask_question(q)
    print("Antwort:", answer)
    print("-" * 300)

The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.



=== llm-2: Feingetuntes Modell ===

Frage: Welche Anforderungen stellt der AI Act an Hochrisiko-KI-Systeme?



The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


Antwort: Welche Anforderungen stellt der AI Act an Hochrisiko-KI-Systeme?
Die AI-Systeme in der Lebensmittelindustrie werden seit Jahren an verschiedenen Stellen in der Lebensmittelindustrie eingesetzt.
Die AI-Systeme bieten einen sehr guten Überblick über die gesamten Anforderungen in Bezug auf Sicherheit, Zuverlässigkeit und Flexibilität.
Als die erste Anwendung wurde im Jahr 2001 die AI-Software für die Lebensmittelindustrie entwickelt.
Mittlerweile kann das System im Bereich der Lebensmittel-Technologie bereits in ca. 100 verschiedenen Anwendungen eingesetzt werden.
Im Folgenden werden einige der größten Herausforderungen der Lebensmittelindustrie umrissen, die eine weitere Anwendung erfordern.
Es gibt ein grundlegendes Problem, das eine Reihe von Anwendungen erfordert.
Das Problem der Zuverlässigkeit ist der Grund dafür, dass Produkte in
-------------------------------------------------------------------------------------------------------------------------------------------------